In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
import cvxopt
from collections import Counter
from sklearn.metrics import confusion_matrix

## 1' Import Dataset

In [2]:
train = pd.concat([pd.read_csv("X_labeled.csv"), pd.read_csv("y_labeled.csv")],1,names =None)
train = train.drop(train.columns[[0,56]],1)

print(train.shape)
print(Counter(train["target"]))
print(Counter(train["school"]))

(196, 56)
Counter({1: 118, -1: 78})
Counter({'GP': 118, 'MS': 78})


## 2' Define SVM

In [3]:
def SVM(train,C_pos,C_neg):
    
    train_pos = train[train["target"]==1]
    train_neg = train[train["target"]==-1]
    
    C_pos = C_pos*np.ones(train_pos.shape[0])
    C_neg = C_neg*np.ones(train_neg.shape[0])
    
    Yt_pos = train_pos.iloc[:,-1]  
    Xt_pos = train_pos.iloc[:,1:-1] 
    
    Yt_neg = train_neg.iloc[:,-1]  
    Xt_neg = train_neg.iloc[:,1:-1]
    
    n_pos,F = Xt_pos.shape
    n_neg,F = Xt_neg.shape
    
    Xt_pos = np.matrix(Xt_pos)
    Yt_pos = np.array(Yt_pos)
    Xt_neg = np.matrix(Xt_neg)
    Yt_neg = np.array(Yt_neg)
    
    w = cp.Variable(F)
    e_pos = cp.Variable(n_pos)
    e_neg = cp.Variable(n_neg)
    b = cp.Variable()
    
    objective = cp.Minimize(0.5*cp.square(cp.norm(w))+
                            cp.sum(cp.multiply(C_pos,e_pos))+
                            cp.sum(cp.multiply(C_neg,e_neg)))
    
    constraints =[cp.multiply(Yt_pos.T,Xt_pos*w+b) - 1 + e_pos >= 0,e_pos >= 0,
                 cp.multiply(Yt_neg.T,Xt_neg*w+b) - 1 + e_neg >= 0,e_neg >= 0]
   
    prob = cp.Problem(objective,constraints)
    
    result = prob.solve()
    
    return w.value, b.value

## 3' Select Best Parameter C

In [4]:
def train_accuracy(w,b,train):
    countin = 0
    result = (np.matmul(train.iloc[:,1:-1],w)+b)*train.iloc[:,-1]
    for i in range(0,len(result)):
        if result[i] > 0:
            countin+=1
    return countin/train.shape[0]

def cmc(C_1,C_2):
    train_result = []
    w,b = SVM(train,C_1,C_2)
    result = (np.matmul(train.iloc[:,1:-1],w)+b)
    for i in range(0,len(result)):
        if result[i] > 0:
            train_result.append(1)
        else:
            train_result.append(-1)
    return confusion_matrix(train["target"],train_result)

diff_thre = 0.06
lower_thre = 1.75
#upper_thre = 
info = []
candidates = []
for i in 2**np.linspace(-15,15,31):
    for j in 2**np.linspace(-15,15,31):
        temp = cmc(i,j)
        acc_sum = temp[0,0]/sum(temp[0,:]) + temp[1,1]/sum(temp[1,:])
        acc_diff = abs(temp[0,0]/sum(temp[0,:]) - temp[1,1]/sum(temp[1,:]))
        info.append([i,j,acc_sum,acc_diff])
        if acc_sum >lower_thre and acc_diff<diff_thre:
            print("pos:",i,"neg:",j,"acc_sum:",acc_sum,"acc_diff:",acc_diff)
            candidates.append([i,j,acc_sum,acc_diff])
pd.DataFrame(info).to_csv("SVM_info.csv")

pos: 0.00390625 neg: 0.0078125 acc_sum: 1.7913950456323338 acc_diff: 0.022164276401564598
pos: 0.0078125 neg: 0.015625 acc_sum: 1.7913950456323338 acc_diff: 0.022164276401564598
pos: 0.015625 neg: 0.03125 acc_sum: 1.8083441981747066 acc_diff: 0.039113428943937434
pos: 0.03125 neg: 0.0625 acc_sum: 1.829639287266406 acc_diff: 0.034767492394610966
pos: 0.0625 neg: 0.125 acc_sum: 1.8339852238157324 acc_diff: 0.013472403302911773
pos: 0.125 neg: 0.25 acc_sum: 1.8552803129074316 acc_diff: 0.009126466753585305
pos: 0.25 neg: 0.5 acc_sum: 1.859626249456758 acc_diff: 0.012168622338113888
pos: 0.5 neg: 1.0 acc_sum: 1.8850499782703172 acc_diff: 0.013255106475445477
pos: 1.0 neg: 1.0 acc_sum: 1.8891786179921772 acc_diff: 0.04302477183833109
pos: 1.0 neg: 2.0 acc_sum: 1.8850499782703172 acc_diff: 0.013255106475445477
pos: 2.0 neg: 2.0 acc_sum: 1.8891786179921772 acc_diff: 0.04302477183833109
pos: 2.0 neg: 4.0 acc_sum: 1.9063450673620164 acc_diff: 0.00890916992611912
pos: 4.0 neg: 4.0 acc_sum: 1.910

In [13]:
#load in after save
info = pd.DataFrame.from_csv("SVM_info.csv")
#info.iloc[:,2] = np.round(info.iloc[:,2],decimals = 2)
#info.iloc[:,3] = np.round(info.iloc[:,3],decimals = 2)
#reduced_info = info.drop_duplicates(["2","3"])
#selected = reduced_info.loc[(reduced_info.iloc[:,2]==max(info.iloc[:,2]))&(reduced_info.iloc[:,3]<0.3),:]
selected = info.loc[(info.iloc[:,2]==max(info.iloc[:,2]))&(info.iloc[:,3]<0.03),:]
#selected = info.loc[(info.iloc[:,3]<0.03),:]
print(selected.shape)
print(selected)

(10, 4)
          0        1         2         3
802  1024.0   4096.0  1.974576  0.025424
803  1024.0   8192.0  1.974576  0.025424
804  1024.0  16384.0  1.974576  0.025424
805  1024.0  32768.0  1.974576  0.025424
834  2048.0   8192.0  1.974576  0.025424
835  2048.0  16384.0  1.974576  0.025424
836  2048.0  32768.0  1.974576  0.025424
866  4096.0  16384.0  1.974576  0.025424
867  4096.0  32768.0  1.974576  0.025424
898  8192.0  32768.0  1.974576  0.025424


C:\Users\KOU TIANQI\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


## 5' Training Accuracy & CM

In [14]:
result_combo = []
for j in np.arange(0,len(selected)-1):
    count = 0
    C_pos,C_neg = selected.iloc[j,0:2]
    w,b = SVM(train,C_pos,C_neg)
    result = (np.matmul(train.iloc[:,1:-1],w)+b)/abs((np.matmul(train.iloc[:,1:-1],w)+b))
    #result = (np.matmul(train.iloc[:,1:-1],w)+b)*train.iloc[:,-1]
    result_combo.append(result)
    print(j+1,":training confusion matrix:\n",confusion_matrix(train["target"],result))
    print("%f" % ((confusion_matrix(train["target"],result)[0,0]+confusion_matrix(train["target"],result)[1,1])/train.shape[0]),"%")

voted = np.squeeze(np.asarray(sum(np.matrix(result_combo)/abs(np.matrix(result_combo)))))/abs(np.squeeze(np.asarray(sum(np.matrix(result_combo)/abs(np.matrix(result_combo))))))
print("voted confusion matrix:\n",confusion_matrix(train["target"],voted))
print("%f" % ((confusion_matrix(train["target"],voted)[0,0]+confusion_matrix(train["target"],voted)[1,1])/train.shape[0]),"%")
print("-1:","%f" % (confusion_matrix(train["target"],voted)[0,0]/sum(confusion_matrix(train["target"],voted)[0,:])),"%")
print("1:","%f" % (confusion_matrix(train["target"],voted)[1,1]/sum(confusion_matrix(train["target"],voted)[1,:])),"%")

1 :training confusion matrix:
 [[ 78   0]
 [  3 115]]
0.984694 %
2 :training confusion matrix:
 [[ 78   0]
 [  3 115]]
0.984694 %
3 :training confusion matrix:
 [[ 78   0]
 [  3 115]]
0.984694 %
4 :training confusion matrix:
 [[ 78   0]
 [  3 115]]
0.984694 %
5 :training confusion matrix:
 [[ 78   0]
 [  3 115]]
0.984694 %
6 :training confusion matrix:
 [[ 78   0]
 [  3 115]]
0.984694 %
7 :training confusion matrix:
 [[ 78   0]
 [  3 115]]
0.984694 %
8 :training confusion matrix:
 [[ 78   0]
 [  3 115]]
0.984694 %
9 :training confusion matrix:
 [[ 78   0]
 [  3 115]]
0.984694 %
voted confusion matrix:
 [[ 78   0]
 [  3 115]]
0.984694 %
-1: 1.000000 %
1: 0.974576 %


## 7' Test Accuracy LOOCV

In [15]:
result_combo_test = []
for j in np.arange(0,len(selected)-1):
    C_pos,C_neg = selected.iloc[j,0:2]
    result = []
    for i in range(0,train.shape[0]):
        X_test = train.iloc[i,1:-1]
        w,b = SVM(train.drop(train.index[i]),C_pos,C_neg)
        result.append(sum(X_test*w)+b)
    result = np.array(result)/abs(np.array(result))
    result_combo_test.append(result)
    print(j+1,":test confusion matrix:\n",confusion_matrix(train["target"],result))
    print("%f" % ((confusion_matrix(train["target"],result)[0,0]+confusion_matrix(train["target"],result)[1,1])/train.shape[0]),"%")
voted_test = np.squeeze(np.asarray(sum(np.matrix(result_combo_test)/abs(np.matrix(result_combo_test)))))/abs(np.squeeze(np.asarray(sum(np.matrix(result_combo_test)/abs(np.matrix(result_combo_test))))))
print("voted confusion matrix:\n",confusion_matrix(train["target"],voted_test))
print("%f" % ((confusion_matrix(train["target"],voted_test)[0,0]+confusion_matrix(train["target"],voted_test)[1,1])/train.shape[0]),"%")
print("-1:","%f" % (confusion_matrix(train["target"],voted_test)[0,0]/sum(confusion_matrix(train["target"],voted_test)[0,:])),"%")
print("1:","%f" % (confusion_matrix(train["target"],voted_test)[1,1]/sum(confusion_matrix(train["target"],voted_test)[1,:])),"%")

1 :test confusion matrix:
 [[60 18]
 [22 96]]
0.795918 %
2 :test confusion matrix:
 [[60 18]
 [22 96]]
0.795918 %
3 :test confusion matrix:
 [[60 18]
 [22 96]]
0.795918 %
4 :test confusion matrix:
 [[60 18]
 [22 96]]
0.795918 %
5 :test confusion matrix:
 [[61 17]
 [22 96]]
0.801020 %
6 :test confusion matrix:
 [[61 17]
 [22 96]]
0.801020 %
7 :test confusion matrix:
 [[61 17]
 [22 96]]
0.801020 %
8 :test confusion matrix:
 [[61 17]
 [23 95]]
0.795918 %
9 :test confusion matrix:
 [[61 17]
 [23 95]]
0.795918 %
voted confusion matrix:
 [[61 17]
 [23 95]]
0.795918 %
-1: 0.782051 %
1: 0.805085 %
